# Example usage notebook
Objects and functions in this notebook are listed with all paramaters to ilustrate their capabillities. Most of the paramaters have default values in the implementation

## Imports

In [1]:
import os
import sys

sys.path.append(os.getcwd())
os.chdir("..")

import pandas as pd
import mlflow

pd.set_option("display.max_columns", 200)
pd.set_option("display.max_rows", 300)

In [2]:
import os
import json
from typing import Literal
import pandas as pd

from churn_pred.preprocessing.preprocess import PreprocessData
from churn_pred.training.trainer import Trainer
from churn_pred.training.optuna_optimizer import LGBOptunaOptimizer
from churn_pred.training.utils import flatten_dict, get_or_create_experiment

import dill
import numpy as np
from churn_pred.utils import dill_dump, dill_load
from sklearn.model_selection import train_test_split

from pprint import pprint

## Train a testing model 

In [3]:
# 1. get the data
df_pd = pd.read_parquet("data/dataset_auxiliary_features_cleaned.parquet")
df_pd.head()

,CustomerId,CreditScore,Country,Gender,Age,Tenure,Balance (EUR),NumberOfProducts,HasCreditCard,IsActiveMember,EstimatedSalary,Exited,CustomerFeedback_sentiment3,CustomerFeedback_sentiment5,Surname_Country,Surname_Country_region,Surname_Country_subregion,Country_region,Country_subregion,is_native,Country_hemisphere,Country_gdp_per_capita,Country_IncomeGroup,Surname_Country_gdp_per_capita,Surname_Country_IncomeGroup,working_class,stage_of_life,generation
0,15787619,844,France,Male,18,2,160980.03,1,0,0,145936.28,0,neutral,4 stars,Taiwan,Asia,Eastern Asia,Europe,Western Europe,0,northern,57594.03402,High income,32756.00000,None,working_age,teen,gen_z
1,15770309,656,France,Male,18,10,151762.74,1,0,1,127014.32,0,neutral,1 star,United States,Americas,Northern America,Europe,Western Europe,0,northern,57594.03402,High income,76329.58227,High income,working_age,teen,gen_z
2,15569178,570,France,Female,18,4,82767.42,1,1,0,71811.90,0,neutral,2 stars,Russian Federation,Europe,Eastern Europe,Europe,Western Europe,0,northern,57594.03402,High income,34637.76172,Upper middle income,working_age,teen,gen_z
3,15795519,716,Germany,Female,18,3,128743.80,1,0,0,197322.13,0,neutral,2 stars,Russian Federation,Europe,Eastern Europe,Europe,Western Europe,0,northern,66616.02225,High income,34637.76172,Upper middle income,working_age,teen,gen_z
4,15621893,727,France,Male,18,4,133550.67,1,1,1,46941.41,0,positive,1 star,Italy,Europe,Southern Europe,Europe,Western Europe,0,northern,57594.03402,High income,55442.07843,High income,working_age,teen,gen_z


In [4]:
target_col = "Exited"
id_cols = ["CustomerId"]
cat_cols = [
    "Country",
    "Gender",
    "HasCreditCard",
    "IsActiveMember",
    "CustomerFeedback_sentiment3",
    "CustomerFeedback_sentiment5",
    "Surname_Country",
    "Surname_Country_region",
    "Surname_Country_subregion",
    "Country_region",
    "Country_subregion",
    "is_native",
    "Country_hemisphere",
    "Country_IncomeGroup",
    "Surname_Country_IncomeGroup",
    "working_class",
    "stage_of_life",
    "generation",
]
cont_cols = df_pd.drop(
    columns=id_cols + cat_cols + [target_col]
).columns.values.tolist()

In [5]:
df_pd[cat_cols] = df_pd[cat_cols].astype(str)

In [6]:
valid_size = 0.2
test_size = 0.5
random_state = 1
df_train, df_valid = train_test_split(
    df_pd, test_size=valid_size, stratify=df_pd[target_col], random_state=random_state
)
df_valid, df_test = train_test_split(
    df_valid,
    test_size=test_size,
    stratify=df_valid[target_col],
    random_state=random_state,
)

In [7]:
prepare_data = PreprocessData(
    id_cols=id_cols,
    target_col=target_col,
    cat_cols=cat_cols,
    cont_cols=cont_cols,
)
# this should be fitted only on training data
_ = prepare_data.fit(df=df_pd)

In [8]:
optimizer = LGBOptunaOptimizer(
    objective="binary",
    n_class=2,
)

trainer = Trainer(
    cat_cols=prepare_data.cat_cols,
    target_col=prepare_data.target_col,
    id_cols=id_cols,
    objective="binary",
    n_class=2,
    optimizer=optimizer,
    preprocessors=[prepare_data],
)

metrics_dict = trainer.fit(
    df_train=df_train,
    df_valid=df_valid,
    df_test=df_test,
)

[I 2024-05-17 10:13:50,499] A new study created in memory with name: no-name-fa589f64-d8f0-4cbb-b929-f47ba65e30cf
feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]

Training until validation scores don't improve for 50 rounds


feature_fraction, val_score: 0.384886:  14%|#4        | 1/7 [00:03<00:18,  3.09s/it]

Early stopping, best iteration is:
[256]	valid_0's binary_logloss: 0.384886
Training until validation scores don't improve for 50 rounds


feature_fraction, val_score: 0.384886:  29%|##8       | 2/7 [00:09<00:24,  4.93s/it]

...

Early stopping, best iteration is:
[68]	valid_0's binary_logloss: 0.378885
Training until validation scores don't improve for 50 rounds


min_child_samples, val_score: 0.367410:  80%|########  | 4/5 [00:15<00:03,  3.21s/it]

Early stopping, best iteration is:
[97]	valid_0's binary_logloss: 0.396152
Training until validation scores don't improve for 50 rounds


min_child_samples, val_score: 0.367410: 100%|##########| 5/5 [00:17<00:00,  3.58s/it]

Early stopping, best iteration is:
[55]	valid_0's binary_logloss: 0.380478


In [9]:
# save/load trainer
dill_dump("lgbm_trainer.dill", trainer)
trainer = dill_load("lgbm_trainer.dill")

In [10]:
pprint(metrics_dict)

{'cls_report': {'0': {'f1-score': 0.9218559218559219,
                      'precision': 0.8966745843230404,
                      'recall': 0.9484924623115578,
                      'support': 796},
                '1': {'f1-score': 0.6464088397790054,
                      'precision': 0.740506329113924,
                      'recall': 0.5735294117647058,
                      'support': 204},
                'accuracy': 0.872,
                'macro avg': {'f1-score': 0.7841323808174636,
                              'precision': 0.8185904567184822,
                              'recall': 0.7610109370381318,
                              'support': 1000},
                'weighted avg': {'f1-score': 0.8656647171122308,
                                 'precision': 0.8648162602603806,
                                 'recall': 0.872,
                                 'support': 1000}},
 'cm': [[755, 41], [87, 117]],
 'prec_rec_curve': [[0.204, 0.740506329113924, 1.0],
                

In [11]:
pprint(trainer.compute_metrics(df_test, with_dynamic_binary_threshold=True))

{'cls_report': {'0': {'f1-score': 0.9244823386114495,
                      'precision': 0.8971631205673759,
                      'recall': 0.9535175879396985,
                      'support': 796},
                '1': {'f1-score': 0.6536312849162011,
                      'precision': 0.7597402597402597,
                      'recall': 0.5735294117647058,
                      'support': 204},
                'accuracy': 0.876,
                'macro avg': {'f1-score': 0.7890568117638252,
                              'precision': 0.8284516901538178,
                              'recall': 0.7635234998522022,
                              'support': 1000},
                'weighted avg': {'f1-score': 0.8692287236576187,
                                 'precision': 0.8691288569586443,
                                 'recall': 0.876,
                                 'support': 1000}},
 'cm': [[759, 37], [87, 117]],
 'prec_rec_curve': [[0.204, 0.7597402597402597, 1.0],
              

# Mlflow
* tracking
* model registration in Minio

In [12]:
mlflow_host = "10.152.183.54"
mlflow_host_url = "mlflow.mlflow.svc.cluster.local"
mlflow_port = "5000"
os.environ["AWS_ACCESS_KEY_ID"] = "minioadmin"
os.environ["AWS_SECRET_ACCESS_KEY"] = "minioadmin"
os.environ["MLFLOW_S3_ENDPOINT_URL"] = f"http://10.152.183.156:9000"

mlflow.set_tracking_uri("http://" + mlflow_host + ":" + mlflow_port)
experiment_id = get_or_create_experiment("ecovadis_assignment")
mlflow.set_experiment(experiment_id=experiment_id)

<Experiment: artifact_location='s3://mlflow/2', creation_time=1715939157478, experiment_id='2', last_update_time=1715939157478, lifecycle_stage='active', name='ecovadis_assignment', tags={}>

In [13]:
metrics_dict_flattened = flatten_dict(metrics_dict)

In [14]:
# mlflow metrics can be only int, float not list
del metrics_dict_flattened["cm"]
del metrics_dict_flattened["prec_rec_curve"]

In [15]:
run_name = "init_run"
with mlflow.start_run(
    experiment_id=experiment_id, run_name=run_name, nested=True
) as run:
    mlflow.log_params(trainer.optimizer.best)
    mlflow.log_metrics(metrics_dict_flattened)

    # Log tags
    mlflow.set_tags(
        tags={
            "project": "SUCCESS6G",
            "optimizer_engine": "optuna",
            "model_family": "ligtgbm",
            "feature_set_version": 1,
        }
    )
    # Log figure - for future fun
    # mlflow.log_figure(figure=correlation_plot, artifact_file="correlation_plot.png")

    artifact_path = "ecovadis_model"
    registered_model_name = "ecovadis_lgbm_model"
    mlflow.pyfunc.log_model(
        python_model=trainer,
        artifact_path=artifact_path,
        registered_model_name=registered_model_name,
    )
    model_uri = mlflow.get_artifact_uri(artifact_path)
    print(f"Run ID:\n{run.info.run_id}\nModel uri:\n{model_uri}")

2024/05/17 10:18:43 INFO mlflow.types.utils: Unsupported type hint: <class 'pandas.core.frame.DataFrame'>, skipping schema inference
2024/05/17 10:18:43 INFO mlflow.types.utils: Unsupported type hint: <class 'pandas.core.frame.DataFrame'>, skipping schema inference
Registered model 'ecovadis_lgbm_model' already exists. Creating a new version of this model...
2024/05/17 10:18:49 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ecovadis_lgbm_model, version 2


Run ID:
18b4ab4d40bb4476b2cbb70b4b6a72a0
Model uri:
s3://mlflow/2/18b4ab4d40bb4476b2cbb70b4b6a72a0/artifacts/ecovadis_model


Created version '2' of model 'ecovadis_lgbm_model'.


## Screenshots from MLflow GUI (Nodeport service in Kubernetes)

![mlflow_parameters](img/mlflow_parameters.png)
![mlflow_parameters](img/mlflow_metrics.png)

![mlflow_parameters](img/mlflow_artifacts.png)

![mlflow_parameters](img/mlflow_artifacts_registered_model.png)

## Predictions testing

**IMPORTANT**: the predictions are in raw_score format(for testing to see if I get same predictions), i.e. strange numbers and not classes in t

### Example data

In [16]:
df_pd.iloc[:2]

,CustomerId,CreditScore,Country,Gender,Age,Tenure,Balance (EUR),NumberOfProducts,HasCreditCard,IsActiveMember,EstimatedSalary,Exited,CustomerFeedback_sentiment3,CustomerFeedback_sentiment5,Surname_Country,Surname_Country_region,Surname_Country_subregion,Country_region,Country_subregion,is_native,Country_hemisphere,Country_gdp_per_capita,Country_IncomeGroup,Surname_Country_gdp_per_capita,Surname_Country_IncomeGroup,working_class,stage_of_life,generation
0,15787619,844,France,Male,18,2,160980.03,1,0,0,145936.28,0,neutral,4 stars,Taiwan,Asia,Eastern Asia,Europe,Western Europe,0,northern,57594.03402,High income,32756.00000,None,working_age,teen,gen_z
1,15770309,656,France,Male,18,10,151762.74,1,0,1,127014.32,0,neutral,1 star,United States,Americas,Northern America,Europe,Western Europe,0,northern,57594.03402,High income,76329.58227,High income,working_age,teen,gen_z


In [17]:
df_pd.iloc[:2].transpose().to_dict(orient="index")

{'CustomerId': {0: 15787619, 1: 15770309},
 'CreditScore': {0: 844, 1: 656},
 'Country': {0: 'France', 1: 'France'},
 'Gender': {0: 'Male', 1: 'Male'},
 'Age': {0: 18, 1: 18},
 'Tenure': {0: 2, 1: 10},
 'Balance (EUR)': {0: 160980.03, 1: 151762.74},
 'NumberOfProducts': {0: 1, 1: 1},
 'HasCreditCard': {0: '0', 1: '0'},
 'IsActiveMember': {0: '0', 1: '1'},
 'EstimatedSalary': {0: 145936.28, 1: 127014.32},
 'Exited': {0: 0, 1: 0},
 'CustomerFeedback_sentiment3': {0: 'neutral', 1: 'neutral'},
 'CustomerFeedback_sentiment5': {0: '4 stars', 1: '1 star'},
 'Surname_Country': {0: 'Taiwan', 1: 'United States'},
 'Surname_Country_region': {0: 'Asia', 1: 'Americas'},
 'Surname_Country_subregion': {0: 'Eastern Asia', 1: 'Northern America'},
 'Country_region': {0: 'Europe', 1: 'Europe'},
 'Country_subregion': {0: 'Western Europe', 1: 'Western Europe'},
 'is_native': {0: '0', 1: '0'},
 'Country_hemisphere': {0: 'northern', 1: 'northern'},
 'Country_gdp_per_capita': {0: 57594.03402, 1: 57594.03402},

after some notepad magic:
```
{
    "CustomerId": [15787619, 15770309],
    "CreditScore": [844, 656],
    "Country": ["France", "France"],
    "Gender": ["Male", "Male"],
    "Age": [18, 18],
    "Tenure": [2, 10],
    "Balance (EUR)": [160980.03, 151762.74],
    "NumberOfProducts": [1, 1],
    "HasCreditCard": ["0", "0"],
    "IsActiveMember": ["0", "1"],
    "EstimatedSalary": [145936.28, 127014.32],
    "Exited": [0, 0],
    "CustomerFeedback_sentiment3": ["neutral", "neutral"],
    "CustomerFeedback_sentiment5": ["4 stars", "1 star"],
    "Surname_Country": ["Taiwan", "United States"],
    "Surname_Country_region": ["Asia", "Americas"],
    "Surname_Country_subregion": ["Eastern Asia", "Northern America"],
    "Country_region": ["Europe", "Europe"],
    "Country_subregion": ["Western Europe", "Western Europe"],
    "is_native": ["0", "0"],
    "Country_hemisphere": ["northern", "northern"],
    "Country_gdp_per_capita": [57594.03402, 57594.03402],
    "Country_IncomeGroup": ["High income", "High income"],
    "Surname_Country_gdp_per_capita": [32756.0, 76329.58227],
    "Surname_Country_IncomeGroup": ["None", "High income"],
    "working_class": ["working_age", "working_age"],
    "stage_of_life": ["teen", "teen"],
    "generation": ["gen_z", "gen_z"],
}
```

### Trainer

In [18]:
trainer.predict(df=df_pd.iloc[:2].drop(columns=["Exited"]), context={})

,Exited
0,-17.300401
1,-15.238104


### Downloaded Trainer

In [19]:
model_uri

's3://mlflow/2/18b4ab4d40bb4476b2cbb70b4b6a72a0/artifacts/ecovadis_model'

In [20]:
loaded_trainer = mlflow.pyfunc.load_model(model_uri)

In [21]:
loaded_trainer.predict(df_pd.iloc[:2].drop(columns=["Exited"]))

,Exited
0,-17.300401
1,-15.238104


### Downloaded/Served Trainer
* i.e. testing [model locally](https://mlflow.org/docs/latest/deployment/deploy-model-to-kubernetes/tutorial.html?highlight=kserve#step-6-testing-model-serving-locally)

In [22]:
model_uri

's3://mlflow/2/18b4ab4d40bb4476b2cbb70b4b6a72a0/artifacts/ecovadis_model'

In [23]:
! mlflow models serve -m "s3://mlflow/2/18b4ab4d40bb4476b2cbb70b4b6a72a0/artifacts/ecovadis_model" --env-manager local -p 5000

2024/05/17 10:19:38 INFO mlflow.models.flavor_backend_registry: Selected backend for flavor 'python_function'
2024/05/17 10:19:38 INFO mlflow.pyfunc.backend: === Running command 'exec gunicorn --timeout=60 -b 127.0.0.1:5000 -w 1 ${GUNICORN_CMD_ARGS} -- mlflow.pyfunc.scoring_server.wsgi:app'
[2024-05-17 10:19:39 +0000] [8811] [INFO] Starting gunicorn 22.0.0
[2024-05-17 10:19:39 +0000] [8811] [INFO] Listening at: http://127.0.0.1:5000 (8811)
[2024-05-17 10:19:39 +0000] [8811] [INFO] Using worker: sync
[2024-05-17 10:19:39 +0000] [8812] [INFO] Booting worker with pid: 8812
^C
[2024-05-17 10:23:52 +0000] [8811] [INFO] Handling signal: int
[2024-05-17 10:23:52 +0000] [8812] [INFO] Worker exiting (pid: 8812)


output from terminal:

```
root@jupyter-5uperpalo:~# curl -X POST -H "Content-Type:application/json" --data '{"inputs": {"CustomerId": [15787619, 15770309], "CreditScore": [844, 656], "Country": ["France", "France"], "Gender": ["Male", "Male"], "Age": [18, 18], "Tenure": [2, 10], "Balance (EUR)": [160980.03, 151762.74], "NumberOfProducts": [1, 1], "HasCreditCard": ["0", "0"], "IsActiveMember": ["0", "1"], "EstimatedSalary": [145936.28, 127014.32], "CustomerFeedback_sentiment3": ["neutral", "neutral"], "CustomerFeedback_sentiment5": ["4 stars", "1 star"], "Surname_Country": ["Taiwan", "United States"], "Surname_Country_region": ["Asia", "Americas"], "Surname_Country_subregion": ["Eastern Asia", "Northern America"], "Country_region": ["Europe", "Europe"], "Country_subregion": ["Western Europe", "Western Europe"], "is_native": ["0", "0"], "Country_hemisphere": ["northern", "northern"], "Country_gdp_per_capita": [57594.03402, 57594.03402], "Country_IncomeGroup": ["High income", "High income"], "Surname_Country_gdp_per_capita": [32756.0, 76329.58227], "Surname_Country_IncomeGroup": ["None", "High income"], "working_class": ["working_age", "working_age"], "stage_of_life": ["teen", "teen"], "generation": ["gen_z", "gen_z"]}}' http://127.0.0.1:5000/invocations
{"predictions": [{"Exited": -17.300400783182656}, {"Exited": -15.2381037279264}]}
```

# [TBD] Model deployment using Kserve
* https://mlflow.org/docs/latest/deployment/deploy-model-to-kubernetes/tutorial.html?highlight=kserve#step-7-deploying-the-model-to-kserve

**Current ISSUE**: the readiness probe is killing the pod: `Readiness probe failed: Get "http://10.1.4.209:8012/": context deadline exceeded (Client.Timeout exceeded while awaiting headers)`